# LangchainとMatching Engineにたわむれる

このノートブックはVertex AI Matching EngineをLangchainのVectorStoreとして扱うサンプルコードを含めています。
1. データセットを `textembedding-gecko`モデルを利用してEmbeddingする
2. ベクトルデータをMatching Enigneに保存
3. `text-bison`をLangchainのAgentとして、Matching EngineをRetrieverとして利用する

## 事前準備、依存関係の解決

事前にVertex AI Matching EngineのIndexとEndpointを作成し、Indexをプロイしておく必要があります。

In [ ]:
PROJECT_ID = ""
REGION = "us-central1"
INDEX_ID = ""
INDEX_BUTCKET_URI = "gs://..."
ENDPOINT_ID = ""

In [ ]:
%pip install langchain google-cloud-aiplatform

In [ ]:
!curl -o dataset.csv -L https://github.com/GoogleCloudPlatform/python-docs-samples/raw/main/cloud-sql/postgres/pgvector/data/retail_toy_dataset.csv

## Matching Enigneの初期化

データセットを雑にEmbeddingsしておきます(後でMatching Engineに投入します)

In [ ]:
from langchain.document_loaders  import CSVLoader
from langchain.text_splitter import CharacterTextSplitter

CHUNK_SIZE=1024
text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE)

loader = CSVLoader(
    file_path="./dataset.csv",
    csv_args=dict(
        delimiter=",",
    )
)
raw_documents = loader.load()
documents = text_splitter.split_documents(raw_documents)

EmbeddingsしたデータセットをMatching Engineへ保存します

In [ ]:
from langchain.vectorstores import MatchingEngine
from langchain.embeddings import VertexAIEmbeddings

# Embeddingsはデフォルトで textembeddings-gecko がセットされています
embeddings = VertexAIEmbeddings(project=PROJECT_ID)

vector_store = MatchingEngine.from_components(
    embedding=embeddings,
    project_id=PROJECT_ID,
    region=REGION,
    gcs_bucket_name=INDEX_BUTCKET_URI,
    index_id=INDEX_ID,
    endpoint_id=ENDPOINT_ID
)

In [ ]:
# データセットをMatchingEngineへ保存します(少し時間がかかります)
vector_store.add_documents(documents)

In [ ]:
# ベクトル近傍値を取得します
search_results = vector_store.similarity_search(query="soccer, game", include_metadata=True)

## AgentとRetrieverのセットアップ

Retriever(ドキュメント探索のインターフェース)のデータソースとしてMatching Engineを設定します

In [ ]:
from langchain.llms import VertexAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

toy_search_llm = VertexAI(temperature=0.1, max_output_tokens=1024)
retriever=vector_store.as_retriever(search_kwargs={"include_metadata": True})
prompt_template = """Only using the descriptions, please exact the product_name and the product_url, and the summary.

Description: {context}

Begin!!

Awesome Summary:
"""

prompt = PromptTemplate(input_variables=['context'], template=prompt_template)
toy_search = RetrievalQA.from_llm(llm=toy_search_llm, prompt=prompt, retriever=retriever)

In [ ]:
toy_search.run("soccer")

Agent(入力をもとにLLMからChainを実行するためのインターフェース)をセットアップします

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

agent_llm = VertexAI(temperature=0.1, max_output_tokens=1024)

tools = [
    Tool(
        name="Retail Toy QA System",
        func=toy_search.run,
        description="useful for when you need to answer questions about the toys. Input should be comma-separated words, do not input a fully formed question.",
    ),
]
agent = initialize_agent(
    tools, llm=agent_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
result = agent.run("""You are a recommendation system that introduces the product that best matches the user's request. 
You must output both of the product's URL and name. The user's request is enclosed in the backquotes (```).

URL: $PRODUCT_URL
NAME: $PRODUCT_NAME

Begin!!!

Request: ```I love soccer game. Tell me good soccer toys?```
""")